# Variogram Analysis
This notebook performs variogram analysis in Python, replicating the functionality of the provided R script. The analysis includes the variogram cloud, empirical variogram, model fitting, sub-area comparison, directional variograms, and a variogram map.

## Step 1: Import Required Libraries
We import all necessary Python libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import curve_fit

## Step 2: Load the Dataset
We load the dataset `testData123.csv` and inspect its structure.

In [ ]:
# Load dataset
data = pd.read_csv('testData123.csv')
print(data.head())

# Extract coordinates and values
coords = data[['x', 'y']].values
values_co = data['Co'].values

## Step 3: Variogram Cloud
We compute the semivariance for all point pairs and plot the variogram cloud.

In [ ]:
# Compute pairwise distances
distances = squareform(pdist(coords))
semivariance_cloud = (values_co[:, None] - values_co[None, :])**2

# Plot variogram cloud
plt.figure(figsize=(10, 6))
plt.scatter(distances.flatten(), semivariance_cloud.flatten(), s=5, alpha=0.5)
plt.xlabel('Pairwise Distance')
plt.ylabel('Semivariance')
plt.title('Variogram Cloud for Co')
plt.grid()
plt.show()

## Step 4: Empirical Variogram
We calculate the empirical variogram by binning the distances.

In [ ]:
def compute_semivariance(values, distances, max_lag, n_bins):
    bins = np.linspace(0, max_lag, n_bins + 1)
    semivariance = []
    bin_centers = []

    for i in range(len(bins) - 1):
        mask = (distances > bins[i]) & (distances <= bins[i + 1])
        if np.any(mask):
            pairwise_diff = (values[:, None] - values[None, :])**2
            semivariance.append(np.mean(pairwise_diff[mask]) / 2)
            bin_centers.append((bins[i] + bins[i + 1]) / 2)
    return np.array(bin_centers), np.array(semivariance)

# Parameters
max_lag = np.max(distances) / 2
n_bins = 15

# Compute empirical variogram
bin_centers_co, semivariance_co = compute_semivariance(values_co, distances, max_lag, n_bins)

# Plot empirical variogram
plt.figure(figsize=(10, 6))
plt.plot(bin_centers_co, semivariance_co, 'o-', label='Empirical Variogram')
plt.xlabel('Lag Distance')
plt.ylabel('Semivariance')
plt.title('Empirical Variogram for Co')
plt.legend()
plt.grid()
plt.show()

## Step 5: Fit a Spherical Variogram Model
We fit a spherical model to the empirical variogram.

In [ ]:
def spherical_model(h, nugget, sill, range_):
    gamma = np.piecewise(
        h, 
        [h <= range_, h > range_],
        [lambda h: nugget + sill * (1.5 * (h / range_) - 0.5 * (h / range_)**3), lambda h: nugget + sill]
    )
    return gamma

# Fit spherical model
initial_guess = [0, np.max(semivariance_co), np.max(bin_centers_co)]
params, _ = curve_fit(spherical_model, bin_centers_co, semivariance_co, p0=initial_guess)
nugget, sill, range_ = params

# Generate model values
model_values = spherical_model(bin_centers_co, nugget, sill, range_)

# Plot fitted model
plt.figure(figsize=(10, 6))
plt.plot(bin_centers_co, semivariance_co, 'o-', label='Empirical Variogram')
plt.plot(bin_centers_co, model_values, 'r-', label='Fitted Spherical Model')
plt.xlabel('Lag Distance')
plt.ylabel('Semivariance')
plt.title('Fitted Spherical Variogram Model')
plt.legend()
plt.grid()
plt.show()

print(f'Nugget: {nugget:.2f}, Sill: {sill:.2f}, Range: {range_:.2f}')

## Step 6: Compare Variograms for Sub-Areas
We divide the dataset into left and right regions and compute their variograms.

In [ ]:
# Split data
mid_x = np.median(data['x'])
left_area = data[data['x'] <= mid_x]
right_area = data[data['x'] > mid_x]

coords_left, values_left = left_area[['x', 'y']].values, left_area['Co'].values
coords_right, values_right = right_area[['x', 'y']].values, right_area['Co'].values

# Compute variograms
distances_left = squareform(pdist(coords_left))
distances_right = squareform(pdist(coords_right))
bin_centers_left, semivariance_left = compute_semivariance(values_left, distances_left, max_lag, n_bins)
bin_centers_right, semivariance_right = compute_semivariance(values_right, distances_right, max_lag, n_bins)

# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(bin_centers_co, semivariance_co, 'o-', label='Full Area')
plt.plot(bin_centers_left, semivariance_left, 's-', label='Left Area')
plt.plot(bin_centers_right, semivariance_right, '^-', label='Right Area')
plt.xlabel('Lag Distance')
plt.ylabel('Semivariance')
plt.title('Comparison of Variograms')
plt.legend()
plt.grid()
plt.show()

## Step 7: Directional Dependency
We compute directional variograms at four angles (0°, 45°, 90°, 135°).

In [ ]:
def directional_variogram(coords, values, angle, tolerance, max_lag, n_bins):
    bins = np.linspace(0, max_lag, n_bins + 1)
    semivariance = []
    bin_centers = []

    dx = np.subtract.outer(coords[:, 0], coords[:, 0])
    dy = np.subtract.outer(coords[:, 1], coords[:, 1])
    distances = np.sqrt(dx**2 + dy**2)
    angles = np.degrees(np.arctan2(dy, dx)) % 180

    for i in range(len(bins) - 1):
        mask = (distances > bins[i]) & (distances <= bins[i + 1]) & (np.abs(angles - angle) <= tolerance)
        if np.any(mask):
            pairwise_diff = (values[:, None] - values[None, :])**2
            semivariance.append(np.mean(pairwise_diff[mask]) / 2)
            bin_centers.append((bins[i] + bins[i + 1]) / 2)
    return np.array(bin_centers), np.array(semivariance)

# Compute directional variograms
angles = [0, 45, 90, 135]
tolerance = 22.5

plt.figure(figsize=(10, 6))
for angle in angles:
    bin_centers_dir, semivariance_dir = directional_variogram(coords, values_co, angle, tolerance, max_lag, n_bins)
    plt.plot(bin_centers_dir, semivariance_dir, 'o-', label=f'Direction {angle}°')
plt.xlabel('Lag Distance')
plt.ylabel('Semivariance')
plt.title('Directional Variograms')
plt.legend()
plt.grid()
plt.show()